In [1]:
# import libraries
import ast
from typing import List
from os import listdir
import spotipy.util as util
import requests
import spotipy
import pandas as pd
import json

In [2]:
# authorize credientials
username = ''
client_id = ''
client_secret = ''
redirect_uri = 'http://localhost:7777/callback'
scope = 'user-read-recently-played'

# create token
token = token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

In [4]:
# function to get strack id to later use to get more track features thorugh API
def get_id(track_info: str, token: str, artist: str = None) -> str:
    # header for API
    headers = {'Accept': 'application/json', 'Content-Type': 'application/json',
    'Authorization': f'Bearer ' + token,}

    # get track names and params
    track_name = track_info.split("___")[0]
    params = [('q', track_name), ('type', 'track'),]
    artist = track_info.split("___")[-1]
    if artist:
        params.append(('artist', artist))
    
    try:
        response = requests.get('https://api.spotify.com/v1/search', 
                    headers = headers, params = params, timeout = 5)
        json = response.json()
        results = json['tracks']['items']
        first_result = json['tracks']['items'][0]
        # Check if searched artist is in response as the first one isn't
        # necessarily the right one
        if artist:
            for result in results:
                if artist.strip() == result['artists'][0]['name'].strip():
                    track_id = result['id']
                    return track_id
        # If specific artist is not found from results, use the first one
        track_id = first_result['id']
        return track_id
    except:
        return None

In [5]:
# function to get features of a track
def get_features(track_id: str, token: str) -> dict:
    sp = spotipy.Spotify(auth=token)
    try:
        features = sp.audio_features([track_id])
        return features[0]
    except:
        return None

In [6]:
# get features of tracks in streaming history provided in sample data
with open('StreamingHistory.json', encoding='utf8') as f:
    streaming_data = json.load(f)

In [13]:
# get unique tracks in streaming history
unique_tracks = list(set([streaming['trackName'] for streaming in streaming_data]))


In [15]:
 # get features of tracks
 
 all_features = {}

 for track in unique_tracks:
     track_id = get_id(track, token)
     features = get_features(track_id, token)
     if features:
         all_features[track] = features

In [16]:
 # combine tracks with features into new list
 with_features = []
 for track_name, features in all_features.items():
     with_features.append({'name': track_name, **features})

In [17]:
# make result into csv file
df = pd.DataFrame(with_features)
df.to_csv('streaming_history.csv')